In [1]:
import pandas as pd
import os
df = pd.read_csv(os.path.join("dataset", "indonesian-news-title-balanced.csv"))   # kolom: title, label

In [2]:
import re
import json
import pickle
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.preprocessing import LabelEncoder


- `re` → untuk *regex*, dipakai saat cleaning teks (hapus simbol, rapikan spasi).
- `json` → untuk menyimpan konfigurasi model (misal `max_len`, `embedding_dim`, dll) ke file `.json`.
- `pickle` → untuk menyimpan objek Python (seperti `word2idx` dan `label_encoder`) ke file `.pkl`.
- `pandas` → untuk membaca dataset dari file `.csv` (judul + label).
- `Counter` → menghitung frekuensi kata, dipakai untuk membangun vocabulary.
- `LabelEncoder` → mengubah label string (misal `"politik"`) menjadi angka (misal `2`).

In [3]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9\s]", " ", text)   # hapus simbol
    text = re.sub(r"\s+", " ", text).strip()      # rapikan spasi
    return text

- `text.lower()` → mengubah seluruh huruf menjadi lowercase agar konsisten.
- `re.sub(r"[^a-zA-Z0-9\s]", " ", text)` → menghapus simbol, tanda baca, emoji, dan karakter non-alfanumerik dengan menggantinya menjadi spasi.
- `re.sub(r"\s+", " ", text)` → mengganti spasi berlebih menjadi satu spasi.
- `strip()` → menghapus spasi di awal dan akhir teks.
- **Tujuan:** menghasilkan teks yang bersih, rapi, dan siap digunakan untuk tokenisasi atau pembuatan sequence.

In [4]:
counter = Counter()

for title in df['title']:
    cleaned = clean_text(title)
    for word in cleaned.split():
        counter[word] += 1

word2idx = {
    "<PAD>": 0,
    "<UNK>": 1
}

for word, _ in counter.items():
    word2idx[word] = len(word2idx)

os.makedirs("artifacts/vocab", exist_ok=True)
with open("artifacts/vocab/word2idx.pkl", "wb") as f:
    pickle.dump(word2idx, f)

- Kode ini membaca seluruh judul berita, membersihkannya, lalu mengumpulkan semua kata yang muncul.
- Setiap kata dicatat dan dihitung frekuensi kemunculannya untuk membentuk dasar vocabulary.
- Setelah semua kata terkumpul, dibuat dictionary `word2idx` yang memetakan setiap kata ke indeks angka unik.
- Token khusus `<PAD>` dan `<UNK>` disertakan sebagai penanda padding dan kata yang tidak dikenal.
- Vocabulary tersebut kemudian disimpan sebagai artefak ke dalam folder `artifacts/vocab` agar bisa digunakan kembali pada tahap training maupun inference.
- Intinya: kode ini membuat dan menyimpan *vocabulary* awal untuk projek yang dikerjain berbasis LSTM/GRU.

In [5]:
MAX_LEN = 20   # sementara gunakan 20, nanti bisa diubah

def encode_text(text):
    cleaned = clean_text(text)
    tokens = cleaned.split()
    seq = [word2idx.get(tok, word2idx["<UNK>"]) for tok in tokens]

    # pad atau trunc
    if len(seq) < MAX_LEN:
        seq = seq + [word2idx["<PAD>"]] * (MAX_LEN - len(seq))
    else:
        seq = seq[:MAX_LEN]

    return seq

- Kode ini ngubah satu kalimat jadi deretan angka biar bisa dimakan model.
- Pertama teks dibersihin dulu, terus dipotong jadi list kata.
- Setiap kata dicek di `word2idx`; kalau nggak ada, dikasih kode `<UNK>`.
- Hasilnya mungkin kependekan atau kepanjangan, jadi disamain panjangnya:
  - Kalau kurang dari MAX_LEN → ditambahin kode `<PAD>`.
  - Kalau lebih panjang → dipotong biar pas 20 token.
- Intinya: fungsi ini bikin setiap judul punya panjang tetap dan format angka, untuk  dimasukin LSTM/GRU.

In [6]:
X = [encode_text(t) for t in df['title']]
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['category'])
os.makedirs("artifacts/labels", exist_ok=True)
with open("artifacts/labels/label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

os.makedirs("artifacts/dataset", exist_ok=True)
np.save("artifacts/dataset/X.npy", np.array(X))
np.save("artifacts/dataset/y.npy", np.array(y))

- Bagian ini ngubah seluruh judul di dataset jadi bentuk angka lewat `encode_text()`, hasilnya masuk ke `X`.
- Label kategori berita juga diubah dari teks jadi angka biar gampang dipelajari model.
- Setelah itu encoder labelnya disimpan ke file supaya nanti waktu inference bisa balikin angka ke nama kategori aslinya.
- Intinya: ini nyiapin input (X) dan output (y) versi numerik + nyimpen mapping labelnya.

In [7]:
config = {
    "max_len": MAX_LEN,
    "num_classes": len(label_encoder.classes_),
    "vocab_size": len(word2idx),
    "embedding_dim": 128,  # sementara, nanti dipakai training
    "hidden_size": 128     # sementara
}

os.makedirs("artifacts/config", exist_ok=True)
with open("artifacts/config/config.json", "w") as f:
    json.dump(config, f, indent=4)

- Kode ini bikin satu paket konfigurasi yang isinya aturan dasar buat model, seperti panjang input, jumlah kelas, ukuran vocab, dan ukuran embedding/hidden.
- Konfigurasi ini disimpen ke file biar nanti training dan inference bisa pakai setting yang sama, jadi nggak ada mismatch.
- Intinya: ini nyimpan “setting penting proyek” supaya semuanya konsisten dari awal sampai akhir.